In [40]:
# Removed  build/intermediate/koges-xrefs-sparql.csv
datasets="build/intermediate/gecko-xrefs-sparql.csv build/intermediate/gcs-xrefs-sparql.csv build/intermediate/genomics-england-xrefs-sparql.csv build/intermediate/maelstrom-xrefs-sparql.csv build/intermediate/saprin-xrefs-sparql.csv build/intermediate/vukuzazi-xrefs-sparql.csv build/intermediate/gcs-xrefs-sparql.csv build/intermediate/genomics-england-xrefs-sparql.csv build/intermediate/maelstrom-xrefs-sparql.csv build/intermediate/saprin-xrefs-sparql.csv build/intermediate/vukuzazi-xrefs-sparql.csv"
datasets=datasets.split(" ")
datasets = ["../../" + sub for sub in datasets] 
datasets
file_basic_out="../../data/ihcc-mapping-suggestions-zooma-basic.tsv"
file_simplestring_out="../../data/ihcc-mapping-suggestions-zooma-simplestring.tsv"

In [41]:
import os
import pandas as pd
import re
from datetime import datetime
import csv


In [42]:
obo_purl="http://purl.obolibrary.org/obo/"
def obo_id_to_iri(obo_id):
    if obo_id.startswith(obo_purl):
        return obo_id
    else:
        return obo_purl+obo_id.replace(":","_")
    
def compute_string_variants(label):
    label_alpha = re.sub('[^0-9a-zA-Z ]+', ' ', label)
    label_alpha = re.sub('\s+', ' ', label_alpha)
    label_split_numerics_alpha = re.sub('(?<=\d)(?!\d)|(?<!\d)(?=\d)', ' ', label_alpha)
    label_split_numerics_alpha_camel = re.sub("([a-z])([A-Z])","\g<1> \g<2>",label_split_numerics_alpha)
    label_split_numerics_alpha = re.sub('\s+', ' ', label_split_numerics_alpha)
    label_split_numerics_alpha_camel = re.sub('\s+', ' ', label_split_numerics_alpha_camel)
    #label_split_numerics = re.sub('(?<=\d)(?!\d)|(?<!\d)(?=\d)', ' ', label)
    #label_split_numerics = re.sub('\s+', ' ', label_split_numerics)
    return [label_alpha.strip(), label_split_numerics_alpha.strip(),label_split_numerics_alpha_camel.strip()]

def process_label(label):
    return re.sub('\s+', ' ', label)

In [43]:
gecko = pd.read_csv(datasets[0])
df = pd.concat([pd.read_csv(f) for f in datasets[1:] ])
df

,from,to,property,label
0,https://purl.ihccglobal.org/GCS_FoodWatOthVeg,PATO:0001995,http://www.w3.org/2000/01/rdf-schema#label,FoodWatOthVeg
1,https://purl.ihccglobal.org/GCS_Vitamin_B12,GECKO:0000072,http://www.w3.org/2000/01/rdf-schema#label,Vitamin B12
2,https://purl.ihccglobal.org/GCS_TransfusionFlag,GECKO:0000091,http://www.w3.org/2000/01/rdf-schema#label,TransfusionFlag
3,https://purl.ihccglobal.org/GCS_WeightLossIntent,OGMS:0000020,http://www.w3.org/2000/01/rdf-schema#label,WeightLossIntent
4,https://purl.ihccglobal.org/GCS_City,GECKO:0000064,http://www.w3.org/2000/01/rdf-schema#label,City
...,...,...,...,...
410,https://purl.ihccglobal.org/VZ_FUSystolicBp,CMO:0000003,http://www.w3.org/2000/01/rdf-schema#label,Clinical Systolic Blood pressure reading
411,https://purl.ihccglobal.org/VZ_ConcurrMed1,GECKO:0000093,http://www.w3.org/2000/01/rdf-schema#label,Concurrent Medication 1
412,https://purl.ihccglobal.org/VZ_FUGlucose,UBERON:0000178,http://www.w3.org/2000/01/rdf-schema#label,Follow Up Glucose
413,https://purl.ihccglobal.org/VZ_EDT2Collected,GECKO:0000112,http://www.w3.org/2000/01/rdf-schema#label,EDT2 Collected


In [44]:
#STUDY,BIOENTITY,PROPERTY_TYPE,PROPERTY_VALUE,SEMANTIC_TAG,ANNOTATOR,ANNOTATION_DATE
# , ,entity,blood,http://purl.obolibrary.org/obo/CMO_0000012,Knocean,2020-08-26 17:00:00
zooma_file_basic = []
zooma_file_simplestring = [] 
current_date=datetime.now().strftime("%Y-%m-%d %H:%M:%S")

for index, row in df.iterrows():
    fromE = row['from']
    toE = obo_id_to_iri(row['to'])
    propertyE = row['property']
    label = process_label(row['label'])
    zooma_file_basic.append(["IHCC Basic",fromE,"information_entity",label,toE,"Knocean",current_date])
    zooma_file_simplestring.append(["IHCC Simple String",fromE,"information_entity",label,toE,"Knocean",current_date])
    string_variants = compute_string_variants(label)
    for string_variant in string_variants:
        zooma_file_simplestring.append(["IHCC Simple String",fromE,"information_entity",string_variant,toE,"Knocean",current_date])

df_basic_out=pd.DataFrame(zooma_file_basic,columns=['STUDY','BIOENTITY','PROPERTY_TYPE','PROPERTY_VALUE','SEMANTIC_TAG','ANNOTATOR','ANNOTATION_DATE'])
df_simplestring_out=pd.DataFrame(zooma_file_simplestring,columns=['STUDY','BIOENTITY','PROPERTY_TYPE','PROPERTY_VALUE','SEMANTIC_TAG','ANNOTATOR','ANNOTATION_DATE'])

df_simplestring_out

,STUDY,BIOENTITY,PROPERTY_TYPE,PROPERTY_VALUE,SEMANTIC_TAG,ANNOTATOR,ANNOTATION_DATE
0,IHCC Simple String,https://purl.ihccglobal.org/GCS_FoodWatOthVeg,information_entity,FoodWatOthVeg,http://purl.obolibrary.org/obo/PATO_0001995,Knocean,2020-08-27 20:37:04
1,IHCC Simple String,https://purl.ihccglobal.org/GCS_FoodWatOthVeg,information_entity,FoodWatOthVeg,http://purl.obolibrary.org/obo/PATO_0001995,Knocean,2020-08-27 20:37:04
2,IHCC Simple String,https://purl.ihccglobal.org/GCS_FoodWatOthVeg,information_entity,FoodWatOthVeg,http://purl.obolibrary.org/obo/PATO_0001995,Knocean,2020-08-27 20:37:04
3,IHCC Simple String,https://purl.ihccglobal.org/GCS_FoodWatOthVeg,information_entity,Food Wat Oth Veg,http://purl.obolibrary.org/obo/PATO_0001995,Knocean,2020-08-27 20:37:04
4,IHCC Simple String,https://purl.ihccglobal.org/GCS_Vitamin_B12,information_entity,Vitamin B12,http://purl.obolibrary.org/obo/GECKO_0000072,Knocean,2020-08-27 20:37:04
...,...,...,...,...,...,...,...
19715,IHCC Simple String,https://purl.ihccglobal.org/VZ_EDT2Collected,information_entity,EDT 2 Collected,http://purl.obolibrary.org/obo/GECKO_0000112,Knocean,2020-08-27 20:37:04
19716,IHCC Simple String,https://purl.ihccglobal.org/VZ_RadiologistLung...,information_entity,Radiologist LungFields,http://purl.obolibrary.org/obo/CMO_0000000,Knocean,2020-08-27 20:37:04
19717,IHCC Simple String,https://purl.ihccglobal.org/VZ_RadiologistLung...,information_entity,Radiologist LungFields,http://purl.obolibrary.org/obo/CMO_0000000,Knocean,2020-08-27 20:37:04
19718,IHCC Simple String,https://purl.ihccglobal.org/VZ_RadiologistLung...,information_entity,Radiologist LungFields,http://purl.obolibrary.org/obo/CMO_0000000,Knocean,2020-08-27 20:37:04


In [45]:
df_basic_out.drop_duplicates().to_csv(file_basic_out,index=False, sep="\t", quoting=csv.QUOTE_NONE, escapechar="\\")

In [46]:
df_simplestring_out.drop_duplicates().to_csv(file_simplestring_out,index=False, sep="\t", quoting=csv.QUOTE_NONE, escapechar="\\")

In [102]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer


In [103]:
X = df_simplestring_out.PROPERTY_VALUE
X = X.str.replace('[\d_]+','')
y = df_simplestring_out.SEMANTIC_TAG
X

0                  FoodWatOthVeg
1                  FoodWatOthVeg
2                  FoodWatOthVeg
3               Food Wat Oth Veg
4                      Vitamin B
                  ...           
19715             EDT  Collected
19716     Radiologist LungFields
19717     Radiologist LungFields
19718     Radiologist LungFields
19719    Radiologist Lung Fields
Name: PROPERTY_VALUE, Length: 19720, dtype: object

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100, stratify = y)

In [105]:
y_test

5273     http://purl.obolibrary.org/obo/GECKO_0000006
6241     http://purl.obolibrary.org/obo/GECKO_0000006
1405      http://purl.obolibrary.org/obo/PATO_0001995
8084     http://purl.obolibrary.org/obo/GECKO_0000064
13909    http://purl.obolibrary.org/obo/GECKO_0000006
                             ...                     
4017     http://purl.obolibrary.org/obo/GECKO_0000052
1947      http://purl.obolibrary.org/obo/PATO_0001995
2378     http://purl.obolibrary.org/obo/GECKO_0000006
17709    http://purl.obolibrary.org/obo/GECKO_0000055
12567    http://purl.obolibrary.org/obo/GECKO_0000052
Name: SEMANTIC_TAG, Length: 3944, dtype: object

In [106]:
count_vect = CountVectorizer(ngram_range = (1,2),min_df=2).fit(X_train)
X_train_counts = count_vect.transform(X_train)

In [107]:
X_train_counts.shape

(15776, 4038)

In [108]:
vocabulary = count_vect.vocabulary_

In [109]:
#Get the most frequent words
sum_words = X_train_counts.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vocabulary.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
print(words_freq[:100])

[('of', 1126), ('mhd', 780), ('date', 684), ('and', 501), ('code', 457), ('score', 454), ('medical', 416), ('officer', 395), ('medical officer', 395), ('vv', 358), ('lung', 352), ('medication', 316), ('care', 306), ('to', 297), ('invest', 270), ('diagnostic', 265), ('radiologist', 263), ('start', 257), ('ic', 238), ('or', 222), ('other', 220), ('age', 215), ('number', 205), ('period', 198), ('concurrent', 195), ('concurrent medication', 195), ('diag', 194), ('treatment', 189), ('not', 184), ('residence', 184), ('collected', 181), ('kr', 181), ('right', 180), ('left', 180), ('id', 176), ('right lung', 171), ('left lung', 171), ('rec', 169), ('ic rec', 169), ('start date', 167), ('hr', 167), ('the', 166), ('cause', 163), ('type', 159), ('treat', 159), ('status', 158), ('icd', 158), ('multiple', 156), ('test', 153), ('high', 152), ('duration', 147), ('flag', 147), ('patient', 146), ('opdate', 146), ('opertn', 145), ('cause code', 144), ('honos', 140), ('icd multiple', 140), ('multiple cau

In [110]:
print(words_freq[-100:])

[('samp hair', 2), ('hair avail', 2), ('mhddischreason', 2), ('regimenmodstoppedearly', 2), ('icrecrpddiscdays', 2), ('interviewpdatey', 2), ('mhdrefrecdate', 2), ('mhdexpirytime', 2), ('txflag', 2), ('reflux heartburn', 2), ('heartburn txflag', 2), ('mhdemerpsychdate', 2), ('loss intent', 2), ('icrecendrprecallflag', 2), ('mhddischmet', 2), ('mhdvalidpostcodeflag', 2), ('jaundice dx', 2), ('liver dx', 2), ('qbleeding', 2), ('mhdmhmdspersonid', 2), ('patientpseudoid', 2), ('mhdexpirydate', 2), ('qsurgerydate', 2), ('bdatepdatem', 2), ('druggroup', 2), ('partnership', 2), ('partnership status', 2), ('degrees', 2), ('temp degrees', 2), ('mhdhonoshonosratingtype', 2), ('friend cell', 2), ('liver age', 2), ('mhdhonossecureratingcscore', 2), ('health status', 2), ('mem area', 2), ('startdateofregimen', 2), ('icspellrpdays', 2), ('last name', 2), ('friend name', 2), ('mhdjobrole', 2), ('oth fish', 2), ('marriage', 2), ('marriage date', 2), ('mhdcodeofcommissioner', 2), ('mhdpostcodedistrict'

In [111]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 0.1).fit(X_train_counts, y_train)

In [112]:
from sklearn.metrics import accuracy_score
X_test_counts = count_vect.transform(X_test)
y_pred  = clf.predict(X_test_counts)

In [145]:
X_test_counts

<3944x4038 sparse matrix of type '<class 'numpy.int64'>'
	with 15824 stored elements in Compressed Sparse Row format>

In [113]:
def print_results(y_test,y_pred):
    acc = accuracy_score(y_test,y_pred)
    print('Overall accuracy : '+str(acc))
    f1_micro = f1_score(y_test, y_pred, average = 'micro')
    f1_macro = f1_score(y_test, y_pred, average = 'macro')
    f1_all = f1_score(y_test, y_pred, average = None)

    print('F1 micro : ' +str(f1_micro))
    print('F1 macro : ' +str(f1_macro))
    print('F1 for each class : ' +str(f1_all))

In [115]:
print_results(y_test,y_pred)

Overall accuracy : 0.9290060851926978
F1 micro : 0.9290060851926978
F1 macro : 0.8940365406840922
F1 for each class : [0.80784314 1.         0.97727273 0.6        1.         1.
 1.         1.         0.95192308 1.         0.75121951 1.
 0.92177077 0.87234043 1.         0.92857143 0.86363636 1.
 0.95652174 0.         0.9787234  1.         0.98       1.
 1.         0.97014925 0.86904762 0.97435897 0.99009901 0.94915254
 0.91803279 1.         1.         1.         0.96385542 0.90956072
 0.96801706 1.         0.97087379 0.94285714 1.         0.66666667
 1.         0.90322581 1.         1.         0.95769882 0.84
 0.7        0.96245734 0.98130841 0.83333333 0.89272031 0.9047619
 0.96875    0.33333333 0.        ]


###### TFIDF

In [118]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(ngram_range = (1,2), stop_words = 'english', min_df = 2).fit(X_train)
X_train_tfidf = tfidf_vect.transform(X_train)
X_train_tfidf

<15776x3694 sparse matrix of type '<class 'numpy.float64'>'
	with 55761 stored elements in Compressed Sparse Row format>

In [121]:
y_train.shape

(15776,)

In [122]:
X_train_tfidf.shape

(15776, 3694)

In [123]:
X_test.shape

(3944,)

In [126]:
y_test.shape

(3944,)

In [144]:
X_test_tfidf

<3944x3694 sparse matrix of type '<class 'numpy.float64'>'
	with 13819 stored elements in Compressed Sparse Row format>

In [130]:
clf2 = MultinomialNB(alpha = 0.1).fit(X_train_tfidf, y_train)
X_test_tfidf = tfidf_vect.transform(X_test)
y_pred  = clf2.predict(X_test_tfidf)
acc = accuracy_score(y_test,y_pred)

In [131]:
X_test_tfidf

<3944x3694 sparse matrix of type '<class 'numpy.float64'>'
	with 13819 stored elements in Compressed Sparse Row format>

In [132]:
print_results(y_test,y_pred)

Overall accuracy : 0.9406693711967545
F1 micro : 0.9406693711967545
F1 macro : 0.8917317559948479
F1 for each class : [0.80784314 1.         0.97727273 0.66666667 1.         1.
 1.         1.         0.96116505 1.         0.75862069 1.
 0.94117647 0.84782609 1.         0.92857143 0.9047619  1.
 0.95614035 0.         0.9787234  1.         1.         1.
 1.         0.97777778 0.94193548 1.         0.98492462 0.95918367
 0.95238095 1.         1.         1.         1.         0.91919192
 0.97674419 1.         0.99009901 0.97058824 0.8        0.66666667
 1.         0.96551724 1.         1.         0.95769882 0.86021505
 0.625      0.97260274 0.98148148 0.8        0.89811321 0.92307692
 0.97674419 0.         0.        ]


###### SVM classifier (linear with stochastic gradient descent for faster computation)

In [133]:
from sklearn.linear_model import SGDClassifier
clf_svg = SGDClassifier(loss='hinge', penalty='l2', alpha = 1e-05).fit(X_train_tfidf,y_train)
y_pred_svg  = clf_svg.predict(X_test_tfidf)

In [134]:
print_results(y_test,y_pred_svg)

Overall accuracy : 0.9523326572008114
F1 micro : 0.9523326572008114
F1 macro : 0.9283239979542038
F1 for each class : [0.8125     1.         0.97727273 0.66666667 1.         1.
 1.         1.         0.98039216 1.         0.75621891 1.
 0.95825985 0.87058824 1.         0.92857143 0.93023256 1.
 0.97368421 1.         0.9787234  1.         1.         1.
 1.         0.97777778 0.98666667 1.         0.99502488 0.95205479
 0.95238095 1.         1.         0.97368421 1.         0.92708333
 0.9875     1.         1.         1.         1.         0.66666667
 1.         0.96428571 1.         1.         0.97133221 0.9
 0.7        0.98786828 0.98617512 0.8        0.90285714 0.95
 1.         0.5        0.        ]


In [135]:
#check accuracy in training set to do a sanity check for overfitting
pred_train = clf_svg.predict(X_train_tfidf)
acc_train = accuracy_score(y_train,pred_train)
print(acc_train)

0.9709685598377282


###### Stemming

In [137]:
from gensim.parsing.porter import PorterStemmer

p = PorterStemmer()

for i in range(len(X)):
    X.iloc[i] = p.stem_sentence(X.iloc[i])
print(X.head())

0       foodwatothveg
1       foodwatothveg
2       foodwatothveg
3    food wat oth veg
4           vitamin b
Name: PROPERTY_VALUE, dtype: object


In [138]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=100)
count_vectorize = CountVectorizer(ngram_range = (1,2), stop_words = 'english', min_df = 2)

tfidf_transform = TfidfTransformer()

pipeline_svm = Pipeline([('bow', count_vectorize),('tfidf', tfidf_transform),('clf_svg', 
                              SGDClassifier(loss='hinge', penalty='l2', alpha = 1e-05))])
#pipeline_svm = Pipeline([('bow', count_vectorize),('tfidf', tfidf_transform),('clf_svm', SVC(kernel = 'rbf'))])

clf_stem = pipeline_svm.fit(X_train, y_train)
y_pred_stem  = clf_stem.predict(X_test)

In [139]:
print_results(y_test,y_pred_stem)

Overall accuracy : 0.9563894523326572
F1 micro : 0.9563894523326572
F1 macro : 0.9246495899416944
F1 for each class : [0.87096774 1.         0.97297297 1.         0.91666667 0.8
 1.         1.         0.97247706 1.         0.79012346 1.
 0.96270396 0.80851064 1.         0.88888889 0.8        0.99259259
 1.         1.         1.         1.         1.         0.98795181
 0.96629213 1.         1.         0.97142857 0.92307692 1.
 1.         1.         1.         0.94949495 0.97321429 1.
 1.         1.         0.55555556 1.         0.88888889 1.
 1.         0.96774194 0.94736842 0.8        0.98648649 0.99173554
 0.6        0.9047619  0.94117647 1.         0.8        0.        ]


In [150]:
class YClass( object ):
    pass

args = YClass()
setattr(args,'config_file','/Users/matentzn/knocean/data-harmonization/src/mapping-suggest/mapping-suggest-config.yml')
setattr(args,'tsv_path','/Users/matentzn/knocean/data-harmonization/build/intermediate/koges-xrefs-sparql.csv')
setattr(args,'tsv_out_path','/Users/matentzn/knocean/data-harmonization/templates/_koges_simple.tsv')

In [161]:
tsv=pd.read_csv(args.tsv_path,sep=",")
tsv_terms=tsv['label'].values[2:]
koges_test=[obo_id_to_iri(item) for item in tsv['to'].values[2:]]

koges_tfidf = tfidf_vect.transform(tsv_terms)
koges_pred  = clf2.predict(koges_tfidf)
acc = accuracy_score(koges_test,koges_pred)

In [162]:
print_results(koges_test,koges_pred)

Overall accuracy : 0.2474747474747475
F1 micro : 0.2474747474747475
F1 macro : 0.2365055439650495
F1 for each class : [0.23529412 0.5        0.         0.8        0.         0.
 0.         0.         0.         0.         0.         0.
 0.66666667 0.8        0.         0.         0.34782609 0.
 1.         0.         0.         0.57142857 0.70588235 0.26666667
 0.         0.         0.         0.5        1.         0.22222222
 0.         0.61538462 0.         0.         0.28282828 0.        ]
